In [1]:
import pandas as pd
import numpy as np

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphadia.extraction.utils import reannotate_fragments
from alphabase.psm_reader import psm_reader_provider

In [2]:
diann_output = '/Users/georgwallmann/Documents/data/2022_10_14_brunner_1ng_rep01/report_ms1.tsv'
output_location = '/Users/georgwallmann/Documents/data/testing/2_feature_dev'

In [3]:
# create dense library from diann psm file
reader = psm_reader_provider.get_reader('diann', keep_decoy=True, fdr=1)
reader.import_file(diann_output)

raw_name = reader.psm_df.raw_name.unique()[0]
reader._psm_df = reader.psm_df[reader.psm_df.raw_name == raw_name]
speclib_psm = SpecLibBase(decoy='diann')
speclib_psm.precursor_df = reader.psm_df
print(len(speclib_psm.precursor_df))

# append decoys and recalculate precursor mz
speclib_psm.append_decoy_sequence()
speclib_psm.calc_precursor_mz()
print(len(speclib_psm.precursor_df))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/georgwallmann/Documents/data/2022_10_14_brunner_1ng_rep01/report_ms1.tsv'

In [3]:
1000 * 300000 * 4 /(1024**3)

1.1175870895385742

In [8]:
from peptdeep.pretrained_models import ModelManager
from alphabase.peptide.fragment import get_charged_frag_types

frag_types = get_charged_frag_types(
    ['b','y'], 
    2
)

model_mgr = ModelManager()

model_mgr.nce = 30
model_mgr.instrument = 'timsTOF'

res = model_mgr.predict_all(
    speclib_psm.precursor_df,
    predict_items=['ms2'],
    frag_types = frag_types,
)

2023-01-07 13:58:39> Using multiprocessing with 8 processes ...
2023-01-07 13:58:39> Predicting ms2 ...


100%|██████████| 24/24 [00:08<00:00,  2.73it/s]


In [18]:
speclib_psm._precursor_df = res['precursor_df']
speclib_psm._fragment_mz_df = res['fragment_mz_df']
speclib_psm._fragment_intensity_df = res['fragment_intensity_df']

In [19]:
import os
speclib_psm.save_hdf(os.path.join(output_location, 'speclib.hdf'))